In [21]:
import sqlite3
import urllib.request, urllib.parse, urllib.error
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

url = input('Enter URL: ')
if (len(url) < 1): url = urllib.request.Request('http://www.py4e.com/code3/mbox.txt', headers={'User-Agent': 'Mozilla/5.0'})
fh = urllib.request.urlopen(url, context=ctx)
counts = dict()
for line in fh:
    decoded_line = line.decode("utf-8").strip()
    if not decoded_line.startswith('From: '): continue
    pieces = decoded_line.split()
    domain = pieces[1].split('@')[1]
    counts[domain] = counts.get(domain, 0) + 1

for key, value in counts.items():
    cur.execute('INSERT INTO Counts (org, count) VALUES (?, ?)', (key, value))

conn.commit()

cur.close()


In [ ]:

    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE email = ? ', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (email, count)
                VALUES (?, 1)''', (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])
